In [6]:
#selenium libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException, TimeoutException
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

import time as t
import pandas as pd
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

In [7]:
import requests
import urllib.parse
import os
import shutil
pd.set_option("max_rows", None)

In [8]:
def get_drive_launch(url):
    """
    This function initiates the Chrome browser's driver with the url passed as a parameter and returns the driver instance.
    Parameters:
        url - url of the website
    return:
        driver - driver instance of the chrome browser    
    """
    #creating driver instance
    driver = webdriver.Chrome('./driver/chromedriver.exe')
    driver.maximize_window()
    
    #defining implicit wait
    driver.implicitly_wait(10)
    
    #launching the url
    driver.get(url)
    
    return driver

In [9]:
def scrape_amazon(d, products):
    ama_revs = []
    ama_rats = []
    for product in products:
                d.find_element_by_id('twotabsearchtextbox').send_keys(product)
                d.find_element_by_id('nav-search-submit-button').click()
                plp_url = d.current_url
                prods_elems = d.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//a[@class='a-link-normal a-text-normal']")
                main_window = d.current_window_handle
                for i in range(0, len(prods_elems)):
                    t.sleep(2)
                    try:
                        prods_elems[i].click()
                    except StaleElementReferenceException:
                        prods_elems = d.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//a[@class='a-link-normal a-text-normal']")
                        prods_elems[i].click()
                    t.sleep(2)
                    #get the window handles and switch to the new tab
                    all_windows = d.window_handles
                    d.switch_to.window(all_windows[1])
                    t.sleep(2)
                    try:
                        d.find_element_by_xpath("//a[@data-hook='see-all-reviews-link-foot']").click()
                        t.sleep(2)
                        while True:
                            reviews = d.find_elements_by_xpath("//div[@class='a-row a-spacing-small review-data']")
                            ratings = d.find_elements_by_xpath("//div[@class='a-row a-spacing-small review-data']/..//div[@class='a-row']/a[@class='a-link-normal']")
                            for review, rating in zip(reviews, ratings):
                                ama_revs.append(review.text)
                                ama_rats.append(rating.get_attribute('title').split('.')[0])
                            try:
                                d.find_element_by_xpath("//a[contains(text(),'Next page')]").click()
                                t.sleep(2)
                            except:
                                break
                    except NoSuchElementException:
                        pass
                    d.close()
                    d.switch_to.window(main_window)
    return ama_revs, ama_rats
        

In [10]:
def scrape_flipkart(d, products, no_pages):
    flip_revs = []
    flip_rats = []
    fliprevs = []
    fliprats = []
    d.find_element_by_xpath("//button[contains(text(),'✕')]").click()
    t.sleep(1)
    for product in products:
                d.find_element_by_xpath("//input[@type='text']").send_keys(product)
                d.find_element_by_xpath("//button[@type='submit']").click()
                t.sleep(2)
                plp_url = d.current_url
                prods_elems = d.find_elements_by_xpath("//div[@class='_4rR01T']")
                main_window = d.current_window_handle
                for i in range(0, len(prods_elems)):
                    try:
                        prods_elems[i].click()
                    except StaleElementReferenceException:
                        prods_elems = d.find_elements_by_xpath("//div[@class='_4rR01T']")
                        prods_elems[i].click()
                    t.sleep(2)
                    #get the window handles and switch to the new tab
                    all_windows = d.window_handles
                    d.switch_to.window(all_windows[1])
                    try:
                        d.find_element_by_xpath("//div[@class='_3UAT2v _16PBlm']").click()
                        t.sleep(2)
                        counter = 0
                        while counter < no_pages:
                            reviews = d.find_elements_by_xpath("//div[@class='t-ZTKy']/div/div")
                            ratings = d.find_elements_by_xpath("//div[@class='_3LWZlK _1BLPMq']")
                            read_mores = d.find_elements_by_xpath("//div[@class='t-ZTKy']//span[contains(text(),'READ MORE')]")
                            i = 0
                            for review, rating in zip(reviews, ratings):
                                try:
                                    read_mores[i].click()
                                except (ElementNotInteractableException,ElementClickInterceptedException, NoSuchElementException):
                                    pass
                                i += 1
                                flip_revs.append(review.text)
                                flip_rats.append(rating.text)
                            try:
                                d.find_element_by_xpath("//a/span[contains(text(),'Next')]").click()
                                t.sleep(2)
                                counter += 1
                            except:
                                break
                    except NoSuchElementException:
                        pass
                    d.close()
                    d.switch_to.window(main_window)
    return flip_revs, flip_rats
        

In [11]:
def scrape_snapdeal(d, products):
    snap_revs = []
    snap_rats = []
    snaprevs = []
    snaprats = []
    for product in products:
                d.find_element_by_id('inputValEnter').send_keys(product)
                d.find_element_by_class_name('searchTextSpan').click()
                plp_url = d.current_url
                for i in range(3):
                    try:
                        d.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                        t.sleep(2)
                    except:
                        break
                    
                try:
                    d.find_element_by_xpath("//p[@class='goToTop']").click()
                except:
                    pass
                t.sleep(1)
                
                prods_elems = d.find_elements_by_xpath("//div[@class='product-desc-rating ']//div[@class='rating-stars ']")
                main_window = d.current_window_handle
                for i in range(0, len(prods_elems)):
                    t.sleep(2)
                    try:
                        prods_elems[i].click()
                    except StaleElementReferenceException:
                        prods_elems = d.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//a[@class='a-link-normal a-text-normal']")
                        prods_elems[i].click()
                    t.sleep(2)
                    #get the window handles and switch to the new tab
                    all_windows = d.window_handles
                    d.switch_to.window(all_windows[1])
                    t.sleep(2)
                    try:
                        d.find_element_by_xpath("//span[@class='numbr-review']").click()
                        t.sleep(1)
                        try:
                            d.find_element_by_xpath("//a[@class='btnload LTgray reset-margin']").click()
                            t.sleep(2)
                        except NoSuchElementException:
                            pass
                        except ElementClickInterceptedException:
                            d.find_element_by_xpath("//a[@class='btnload LTgray reset-margin']").click()
                            t.sleep(2)
                        while True:
                            ratings = d.find_elements_by_xpath("//div[@class='commentreview']/div//div[@class='rating']")
                            review_heads = d.find_elements_by_xpath("//div[@class='commentreview']/div//div[@class='head']")
                            review_bodies = d.find_elements_by_xpath("//div[@class='user-review']//p")
                            
                            for rating, review_head, review_body in zip(ratings, review_heads, review_bodies):
                                r = rating
                                stars = r.find_elements_by_xpath(".//i[@class='sd-icon sd-icon-star active']")
                                snap_rats.append(len(stars))
                                snap_revs.append(review_head.text+'.'+review_body.text)
                            
                            try:
                                d.find_element_by_xpath("//i[@class='sd-icon sd-icon-next ']").click()
                                t.sleep(2)
                            except NoSuchElementException:
                                break
                    except NoSuchElementException:
                        pass
                    d.close()
                    d.switch_to.window(main_window)
                    
    return snap_revs, snap_rats
        

In [12]:
def scrape_reviews(sites, product):
    urls = {'amazon':'https://www.amazon.in/',
            'flipkart':'https://www.flipkart.com/',
            'snapdeal':'https://www.snapdeal.com/',
            'croma': 'https://www.croma.com/'}
    products = [product]

    revs = []
    rats = []
    reviews = []
    ratings = []

    for site in sites:
        d = get_drive_launch(urls[site])
        if site.lower() == 'amazon':
            reviews, ratings = scrape_amazon(d, products)
        elif site.lower() == 'flipkart':
            reviews, ratings = scrape_flipkart(d, products,1)
        elif site.lower() == 'snapdeal':
            reviews, ratings = scrape_snapdeal(d, products)
        d.quit()
        revs = revs + reviews
        rats = rats + ratings
    data = pd.DataFrame({'Reviews': revs, 'Ratings':rats})
    return data

In [ ]:
sites=['snapdeal','flipkart','amazon']
#products = ['laptops', 'phones', 'headphones', 'smart watches', 'Professional Cameras', 'Printers', 'monitors', 'Home Theater', 'router']
products = ['Home Theater', 'router']
#sites=['snapdeal']
#products = ['laptops']

for product in products:
    review_data = scrape_reviews(sites, product)
    review_data.to_csv('./data/all_sites_'+product+'_reviews_data.csv', sep = ',', index=False)